# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, WT, Female, VISp, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISp and VISl sessions. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('wt/wt') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISp' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"
739448407,2019-10-03T00:00:00Z,716813543,brain_observatory_1.1,112.0,M,wt/wt,625,2221,6,"[grey, VISam, nan, VIS, VISp, VISl, VISrl]"
742951821,2019-10-03T00:00:00Z,723627604,brain_observatory_1.1,120.0,M,wt/wt,893,2219,6,"[VISal, nan, grey, VISl, VISrl, VISp, VISpm, VIS]"
743475441,2019-10-03T00:00:00Z,722882755,brain_observatory_1.1,121.0,M,wt/wt,553,2225,6,"[LP, LGd, HPF, DG, CA3, CA1, VISrl, nan, PP, P..."


In [7]:
session_id = filtered_sessions.index.values[3]

session = cache.get_session_data(session_id)

Downloading:   0%|          | 0.00/2.92G [00:00<?, ?B/s]

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

/Users/tulsipatel/.local/lib/python3.8/site-packages/allensdk/brain_observatory/ecephys/ecephys_project_api/ecephys_project_warehouse_api.py:291: FutureWarning: Conversion of the second argument of issubdtype from `bool` to `np.generic` is deprecated. In future, it will be treated as `np.bool_ == np.dtype(bool).type`.
  pv_is_bool = np.issubdtype(output["p_value_rf"].values[0], np.bool)


Total number of units: 893


,snr,waveform_recovery_slope,cumulative_drift,waveform_halfwidth,cluster_id,amplitude_cutoff,L_ratio,isi_violations,d_prime,waveform_velocity_below,waveform_spread,waveform_repolarization_slope,waveform_amplitude,peak_channel_id,waveform_PT_ratio,local_index_unit,silhouette_score,presence_ratio,waveform_velocity_above,firing_rate,isolation_distance,max_drift,nn_miss_rate,waveform_duration,nn_hit_rate,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
950920817,3.246479,-0.393127,346.73,0.068677,6,0.061671,0.000078,0.061295,6.236154,0.000000,70.0,0.184384,173.942145,850186862,1.194054,6,0.148733,0.99,-0.274707,20.378176,129.193512,17.98,0.000699,0.109883,0.998000,NaN,300.0,14.019476,0.857436,0.879457,1.420513,1.288028,0.179941,0.098358,0.095574,0.013917,13.333,0.884362,0.911,0.32,8.0,NaN,0.537313,-0.238342,NaN,NaN,225.0,150.0,NaN,0.004315,0.191413,NaN,NaN,-23.333,4996,0.00,18.013558,3.623646,32.136496,3.592125,34.757226,NaN,0.0225,False,False,False,False,False,-0.136017,False,0.040639,0.042337,0.001979,0.073979,0.002895,200,59,769322712,18,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970281,1249.998762,True
950920914,3.409441,-0.089997,123.51,0.123618,11,0.000346,0.000037,0.003499,7.759853,-0.412060,80.0,0.619365,144.634425,850186876,0.410837,11,0.163029,0.99,0.000000,24.031002,100.752378,28.54,0.000000,0.247236,0.998000,NaN,200.0,1.125651,1.085897,1.530000,1.012698,1.492711,0.370398,0.011217,0.003784,0.021493,10.000,6.905333,0.397,0.32,4.0,NaN,0.067568,-0.194268,NaN,-0.093960,135.0,60.0,NaN,0.437496,0.085780,NaN,0.468639,45.000,4928,0.50,24.884868,26.484588,23.120839,26.251242,23.693264,NaN,0.2155,False,False,False,False,False,-0.033136,False,0.001183,0.004274,0.003128,0.008312,0.004566,260,11,769322712,25,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970281,1249.998762,True
950920977,5.953624,-0.034566,483.50,0.178559,14,0.000270,0.000021,0.000000,5.608442,0.412060,50.0,0.753621,282.601215,850186882,0.827002,14,0.334548,0.98,-2.747069,1.422358,79.773528,26.22,0.000101,0.370854,0.991597,NaN,100.0,12.794350,1.762424,4.421538,11.308333,7.829817,0.735024,0.320987,0.231522,0.045168,10.000,0.997451,0.217,0.02,15.0,-1.000000,-1.000000,-1.000000,-1.0,NaN,270.0,90.0,0.009782,0.015274,0.001314,0.272069,NaN,-30.000,4924,0.25,1.051585,0.452956,1.376328,0.495617,1.953678,NaN,0.2255,False,False,False,False,False,0.390559,False,0.446530,0.159236,0.197305,0.678174,0.151251,300,43,769322712,28,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970281,1249.998762,True
950921087,4.651214,-0.159289,87.94,0.109883,20,0.000133,0.000053,0.004755,8.108768,0.000000,70.0,0.862546,200.558670,850186896,0.340643,20,0.185451,0.99,0.343384,31.314676,147.363079,23.49,0.000747,0.206030,1.000000,NaN,600.0,0.577732,0.795444,0.585339,1.668627,0.782749,0.156185,0.007879,0.010648,0.008557,80.000,0.850903,0.695,0.08,8.0,NaN,-0.128114,-0.106599,NaN,-0.151399,225.0,30.0,NaN,0.035289,0.040818,NaN,0.134065,5.000,4982,0.50,30.

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISp') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 18


,snr,waveform_recovery_slope,cumulative_drift,waveform_halfwidth,cluster_id,amplitude_cutoff,L_ratio,isi_violations,d_prime,waveform_velocity_below,waveform_spread,waveform_repolarization_slope,waveform_amplitude,peak_channel_id,waveform_PT_ratio,local_index_unit,silhouette_score,presence_ratio,waveform_velocity_above,firing_rate,isolation_distance,max_drift,nn_miss_rate,waveform_duration,nn_hit_rate,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
950946855,4.383121,-0.041146,87.63,0.151089,397,0.006109,0.000077,0.000000,6.946469,-0.206030,60.0,0.460069,137.930520,850184574,0.345344,391,0.328121,0.96,0.000000,0.449194,75.003416,41.21,0.000269,0.810385,0.974359,NaN,0.0,3.600000,2.236396,3.116522,1.866667,4.600903,1.272360,0.341420,0.499599,0.184078,NaN,1.543844,0.026,0.32,1.0,0.687500,0.777778,-0.911111,NaN,-0.541667,45.0,90.0,0.000010,0.201963,0.000015,NaN,0.403246,NaN,4984,0.75,0.318229,1.092423,0.787913,0.021930,0.456946,NaN,0.1365,False,False,False,False,False,0.784119,False,0.738826,0.786282,0.657636,0.966566,0.525603,2820,11,769322716,281,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.998985,1249.999958,True
950949898,2.761324,-0.018220,132.03,0.192295,573,0.001915,0.010974,0.020850,3.753579,-0.961474,70.0,0.173260,69.726930,850184576,0.374028,567,0.009011,0.99,0.412060,3.766240,60.387631,47.15,0.008959,0.796650,0.912000,NaN,700.0,2.523656,2.885735,3.295528,3.400000,3.441818,0.658824,0.011356,0.312047,0.326321,14.286,0.697350,0.001,0.02,15.0,0.608791,0.672857,0.357525,0.734266,0.549550,315.0,30.0,0.000002,0.000042,0.218493,0.002623,0.145689,17.143,4999,0.25,3.951652,10.711071,4.030244,3.384887,3.812770,NaN,0.1475,False,False,False,False,False,0.494915,False,0.195793,0.261762,0.305319,0.154306,0.397379,2840,59,769322716,282,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.998985,1249.999958,True
950947261,2.922073,-0.075110,139.86,0.109883,422,0.004068,0.008545,0.016954,4.729724,-0.274707,90.0,0.322369,72.063420,850184610,0.529977,416,0.098105,0.99,-0.137353,11.383374,86.524692,49.31,0.002180,0.206030,0.984667,NaN,700.0,7.923810,1.816037,3.414268,3.009524,2.576667,0.646780,0.061036,0.078422,0.030857,31.429,1.282228,0.273,0.02,2.0,0.609756,0.503968,0.001057,NaN,-0.045455,225.0,60.0,0.005297,0.117983,0.997360,NaN,0.876651,47.143,4942,0.25,10.251102,15.080761,15.512410,4.695201,14.655586,NaN,0.0585,False,False,False,False,False,0.360966,False,0.047861,0.000225,0.030908,0.141442,0.053776,3000,27,769322716,299,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.998985,1249.999958,True
950947375,1.266898,-0.019769,172.13,0.137353,426,0.005072,0.001466,0.021380,3.286085,-1.373534,110.0,0.175083,41.059395,850184616,0.516869,420,0.031107,0.99,-0.049055,4.662373,66.493469,50.16,0.000343,0.315913,0.973472,NaN,400.0,10.342513,1.884103,4.590427,1.987879,3.210470,0.403790,0.017829,0.179379,0.260106,65.000,1.933388,0.000,0.08,15.0,0.590591,0.761905,0.700413,0.666667,0.436047,90.0,120.0,0.1018

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
950946855     0.456946
950949898     3.812770
950947261    14.655586
950947375     7.580246
950947357    37.096577
950947341     7.672168
950947439     9.974218
950947509     7.211891
950947464     3.479719
950947548     2.914198
950947663     4.667380
950947797     8.921110
950947766     2.648423
950947830    11.560208
950948211    12.236302
950948417     9.519270
950948559     0.385007
950948976     9.748409
Name: firing_rate_sg, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

8.585579465384294


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
950946855     0.318229
950949898     3.951652
950947261    10.251102
950947375     5.191209
950947357    33.878939
950947341     7.093169
950947439     9.584992
950947509    10.831962
950947464     2.328737
950947548     0.792660
950947663     3.783271
950947797     6.803004
950947766     4.191515
950947830     9.205341
950948211     8.617068
950948417     2.398631
950948559     0.187443
950948976     6.886665
Name: firing_rate_dg, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

7.01642154339164


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])